# 1.Cluster Assessment

# 2.Data Preparation

# 3.Feature Engineer

# 4.Data Split

# 5.Model Training

## 5.1.Model Selection

## 5.2.Efficient Model

# 6.Top Feature Extraction and Model Retrain

# 7.Top Feature vs Cluster